# ECE 493 - Probabilistic Reasoning and Decision Making

## Probability

1. *See [Probability and Statistics for Computer Science](http://jacobpark.me/Programming-Notebook/Books/Probability%20and%20Statistics%20for%20Computer%20Science/Probability%20and%20Statistics%20for%20Computer%20Science.html).*
2. *See [Stanford CS229 - Probability Review](https://ermongroup.github.io/cs228-notes/preliminaries/probabilityreview/).*